# Functional test with FTDI
- https://www.silabs.com/documents/public/data-sheets/Si5351-B.pdf
- https://www.silabs.com/documents/public/application-notes/AN619.pdf
- https://www.silabs.com/content/usergenerated/asi/cloud/attachments/siliconlabs/en/community/groups/timing/knowledge-base/jcr:content/content/primary/blog/modifying_the_feedba-K8Pv/311668.pdf

In [6]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [7]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [8]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [9]:
%pylab inline

from clock_generators.si535x.si5351 import Si5351 
from utilities.adapters import peripherals

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [10]:
cls = Si5351

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [15]:
with_hardware_device = True

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

si = cls(_i2c)  

## Registers values file I/O

In [16]:
si.map.save_to_file('Initial_values.txt')
initial_values = si.map.read_file('Initial_values.txt') 

In [14]:
freq = 104e5
# freq = 106e5

si.reset()

for i in range(3):
    si.clocks[i].set_frequency(freq)
    
si.print_registers_values()

# print('registers_values = {}'.format(initial_values))

registers_values = [(0, 0), (1, 0), (2, 0), (3, 0), (9, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 255), (43, 255), (44, 0), (45, 41), (46, 68), (47, 240), (48, 0), (49, 0), (50, 255), (51, 255), (52, 0), (53, 41), (54, 68), (55, 240), (56, 0), (57, 0), (58, 255), (59, 255), (60, 0), (61, 41), (62, 68), (63, 240), (64, 0), (65, 0), (66, 0), (67, 0), (68, 0), (69, 0), (70, 0), (71, 0), (72, 0), (73, 0), (74, 0), (75, 0), (76, 0), (77, 0), (78, 0), (79, 0), (80, 0), (81, 0), (82, 0), (83, 0), (84, 0), (85, 0), (86, 0), (87, 0), (88, 0), (89, 0), (90, 0), (91, 0), (92, 0), (149, 0), (150, 0), (151, 0), (152, 0), (153, 0), (154, 0), (155, 0), (156, 0), (157, 0), (158, 0), (159, 0), (160, 0), (161, 0), (162, 0), (163, 0), (164, 0), (165, 0), (166, 0), (167, 0), (168, 0), (169, 0), (170

In [17]:
# clock_builder_values = si.map.read_file('Si5351A-RevB-Registers.txt') 
clock_builder_values = si.map.read_file('Si5351A-RevB-Registers with CenterSS 6ns skew.txt')

In [ ]:
# si.write_all_registers(initial_values)

In [ ]:
si.map.reset()
si.read_all_registers()
si.map.save_to_file('Read_back_values.txt')
read_back_values = si.map.read_file('Read_back_values.txt')

## Compare Register Value Sets

In [ ]:
si.map.compare_values_sets(clock_builder_values, initial_values)

In [ ]:
si.map.compare_values_sets(initial_values, read_back_values) 

## Find Integer Dividers for a Frequency

In [ ]:
matchs = si.find_integer_dividers(freq_desired = 90e6, even_only = True, torance_hz = 1) 
matchs

In [ ]:
matchs = si.find_integer_dividers(freq_desired = 150e6, even_only = True, torance_hz = 1) 
matchs

## Find Common Integer Dividers for Frequencies

In [ ]:
desired_clock_freqs = (45e6, 30e5, 90e6)
# desired_clock_freqs = (125e6, 48e6, 28.322e6, 74.25e6, 74.25e6, 24.576e6, 22.5792e6, 33.3333e6)
# desired_clock_freqs = (125e6, 48e6, 28.322e6, 74.25e6, 74.25e6, 24.576e6, 22.5792e6, 33.3333e6)
# desired_clock_freqs = (25e6, 25e6)

common_pll_dividers, freqs_pll_dividers, freqs_matches = si.find_integer_pll_dividers_for_clocks(desired_clock_freqs)
common_pll_dividers, freqs_pll_dividers

## Main functions test

In [19]:
for f in dir(cls):
    if not f.startswith('_'):
        print('si.{}()'.format(f))

si.DEBUG_MODE()
si.DEBUG_MODE_PRINT_REGISTER()
si.DEBUG_MODE_SHOW_BUS_DATA()
si.FREQ_REF()
si.HAS_VCXO()
si.I2C_ADDRESS()
si.N_OUTPUT_CLOCKS()
si.OUTPUT_CLOCKS_IN_USE()
si.READ_ONLY_REGISTERS()
si.close()
si.current_frequency()
si.current_phase()
si.do()
si.do_on_devices()
si.enable()
si.enable_output()
si.enable_output_channel()
si.find_integer_dividers()
si.find_integer_pll_dividers_for_clocks()
si.init()
si.is_virtual_device()
si.load_registers()
si.pause()
si.print()
si.print_registers_values()
si.read_all_registers()
si.ready()
si.reset()
si.reset_plls()
si.restore_clocks_freqs()
si.resume()
si.revision()
si.set_frequency()
si.set_phase()
si.start()
si.status()
si.stop()
si.update()
si.write_all_registers()


In [20]:
si.init()

In [21]:
si.is_virtual_device

False

In [22]:
si.revision

1

In [23]:
# si.set_frequency(2323)

In [24]:
# si.set_phase(55)

In [25]:
si.reset()

In [26]:
# si.current_frequency

In [27]:
# si.current_phase

In [28]:
si.enable()

In [29]:
si.enable_output()

In [30]:
si.enable_output_channel(0)

In [31]:
si.load_registers(initial_values)

In [32]:
si.read_all_registers()

[(0, 'Device_Status', 17),
 (1, 'Interrupt_Status_Sticky', 240),
 (2, 'Interrupt_Status_Mask', 0),
 (3, 'Output_Enable_Control', 0),
 (9, 'OEB_Pin_Enable_Control_Mask', 0),
 (15, 'PLL_Input_Source', 0),
 (16, 'CLK0_Control', 0),
 (17, 'CLK1_Control', 0),
 (18, 'CLK2_Control', 0),
 (19, 'CLK3_Control', 0),
 (20, 'CLK4_Control', 0),
 (21, 'CLK5_Control', 0),
 (22, 'CLK6_Control', 0),
 (23, 'CLK7_Control', 0),
 (24, 'CLK3_0_Disable_State', 0),
 (25, 'CLK7_4_Disable_State', 0),
 (26, 'Multisynth_NA_Parameters_26', 0),
 (27, 'Multisynth_NA_Parameters_27', 0),
 (28, 'Multisynth_NA_Parameters_28', 0),
 (29, 'Multisynth_NA_Parameters_29', 0),
 (30, 'Multisynth_NA_Parameters_30', 0),
 (31, 'Multisynth_NA_Parameters_31', 0),
 (32, 'Multisynth_NA_Parameters_32', 0),
 (33, 'Multisynth_NA_Parameters_33', 0),
 (34, 'Multisynth_NB_Parameters_34', 0),
 (35, 'Multisynth_NB_Parameters_35', 0),
 (36, 'Multisynth_NB_Parameters_36', 0),
 (37, 'Multisynth_NB_Parameters_37', 0),
 (38, 'Multisynth_NB_Paramete

In [33]:
si.ready

True

In [34]:
si.status.print()


<< Device_Status >>:  ('0x11', '0b10001')
[ SYS_INIT ]   :  0
[ LOL_B ]      :  0
[ LOL_A ]      :  0
[ LOS_CLKIN ]  :  1
[ LOS_XTAL ]   :  0
[ Reserved_2 ] :  0
[ REVID ]      :  1


10

In [35]:
si.restore_clocks_freqs()

In [36]:
si.write_all_registers()

In [37]:
si.start()

In [38]:
si.pause()

In [39]:
si.resume()

In [40]:
si.update()

In [41]:
si.stop()

In [42]:
si.close()

In [43]:
si.print()


<< Device_Status >>:  ('0x11', '0b10001')
[ SYS_INIT ]   :  0
[ LOL_B ]      :  0
[ LOL_A ]      :  0
[ LOS_CLKIN ]  :  1
[ LOS_XTAL ]   :  0
[ Reserved_2 ] :  0
[ REVID ]      :  1

<< Interrupt_Status_Sticky >>:  ('0xf0', '0b11110000')
[ SYS_INIT_STKY ]  :  1
[ LOL_B_STKY ]     :  1
[ LOL_A_STKY ]     :  1
[ LOS_CLKIN_STKY ] :  1
[ LOS_XTAL_STKY ]  :  0
[ Reserved_0 ]     :  0

<< Interrupt_Status_Mask >>:  ('0x0', '0b0')
[ SYS_INIT_MASK ]   :  0
[ LOL_B_MASK ]      :  0
[ LOL_A_MASK ]      :  0
[ LOS__CLKIN_MASK ] :  0
[ LOS__XTAL_MASK ]  :  0
[ Reserved_0 ]      :  0

<< Output_Enable_Control >>:  ('0xff', '0b11111111')
[ CLK7_OEB ] :  1
[ CLK6_OEB ] :  1
[ CLK5_OEB ] :  1
[ CLK4_OEB ] :  1
[ CLK3_OEB ] :  1
[ CLK2_OEB ] :  1
[ CLK1_OEB ] :  1
[ CLK0_OEB ] :  1

<< OEB_Pin_Enable_Control_Mask >>:  ('0x0', '0b0')
[ OEB_MASK_7 ] :  0
[ OEB_MASK_6 ] :  0
[ OEB_MASK_5 ] :  0
[ OEB_MASK_4 ] :  0
[ OEB_MASK_3 ] :  0
[ OEB_MASK_2 ] :  0
[ OEB_MASK_1 ] :  0
[ OEB_MASK_0 ] :  0

<< PLL_Inp

## Spread Spectrum test

In [57]:
si.init()

In [70]:
si.spread_spectrum.enable(True, mode = 'center', ssc_amp = 0.01)

In [71]:
si.spread_spectrum.enable(True, mode = 'down', ssc_amp = 0.01)

In [72]:
si.spread_spectrum.enable(False)

In [50]:
clk = si.clocks[0]